# State space representation

The "standard" or most commonly used state space representation is 

\begin{align} \dot{x} &= Ax + Bu \\ y &= Cx + Du \end{align}

Take note that Seborg uses a slightly different version:

\begin{align} \dot{x} &= Ax + Bu + Ed\\ y &= Cx \end{align}

This second version can not represent pure gain systems as it effectively assumes $D=0$. It is also possible to stack $u$ and $d$ from the bottom form into one input vector, so the $E$ matrix really doesn't add much. As you may infer, I prefer the top version and it is also the version used by most libraries.

In [1]:
import numpy
import numpy.linalg

## Converting between state space and transfer function forms

There is good support in various libraries for converting systems with numeric coefficients between transfer function and state space representation.

### Scipy.signal

The `scipy.signal` library handles conversion between transfer function coefficients and state space matrices easily. Note that `scipi.signal` only handles SISO transfer functions.

In [2]:
import scipy.signal

In [3]:
G = scipy.signal.lti(1, [1, 1])

In [4]:
G

TransferFunctionContinuous(
array([1.]),
array([1., 1.]),
dt: None
)

This object allows us to access the numerator and denominator

In [5]:
G.num, G.den

(array([1.]), array([1., 1.]))

To convert to state space, we can use the `.to_ss()` method

In [6]:
Gss = G.to_ss()

In [7]:
Gss.A, Gss.B, Gss.C, Gss.D

(array([[-1.]]), array([[1.]]), array([[1.]]), array([[0.]]))

We can build another object using the state space matrices instead of the Laplace form

In [8]:
G2ss = scipy.signal.lti(Gss.A, Gss.B, Gss.C, Gss.D)
G2ss

StateSpaceContinuous(
array([[-1.]]),
array([[1.]]),
array([[1.]]),
array([[0.]]),
dt: None
)

We can convert to transfer function form using `.to_tf()` (there is a small warning about bad coefficients, but the answer is reliable).

In [9]:
G2 = G2ss.to_tf()

C:\Users\Admin\anaconda3\lib\site-packages\scipy\signal\filter_design.py:1630: BadCoefficients: Badly conditioned filter coefficients (numerator): the results may be meaningless
  warnings.warn("Badly conditioned filter coefficients (numerator): the "


We can now access the numerator and denominator again:

In [10]:
G2.num, G2.den

(array([1.]), array([1., 1.]))

Instead of building objects we can also use the functions in `scipy.signal.lti_conversion`:

In [11]:
scipy.signal.lti_conversion.tf2ss(1, [1, 1])

(array([[-1.]]), array([[1.]]), array([[1.]]), array([[0.]]))

In [12]:
scipy.signal.lti_conversion.ss2tf(-1, 1, 1, 0)

(array([[0., 1.]]), array([1., 1.]))

### Control library

The control library (at least from version 0.8.0) does a good job with these conversions as well.

In [13]:
import control

In [14]:
Gtf = control.tf([1], [1, 1])
Gtf

TransferFunction(array([1]), array([1, 1]))

In the control library we convert the system using `ss` (short for state space) to get a State Space representation:

In [15]:
Gss = control.ss(Gtf)
Gss

StateSpace(array([[-1.]]), array([[1.]]), array([[1.]]), array([[0.]]))

In [16]:
Gss.A

array([[-1.]])

## Symbolic conversion

It is easy to convert state space models to transfer functions since the Laplace transform is a linear operator:

$$ \dot{x} = Ax + Bu \quad \therefore \quad sX(s) = AX(s) + BU(s) \quad X(s) = (sI - A)^{-1}BU(s)$$
$$ y = Cx + Du \quad \therefore \quad Y(s) = CX(s) + DU(s) \quad Y(s) = \underbrace{(C(sI - A)^{-1}B + D)}_{G(s)}U(s)$$

This conversion is handled for symbolic matrices by `tbcontrol.symbolic.ss2tf`

In [17]:
import sympy

In [18]:
import tbcontrol
tbcontrol.expectversion('0.1.8')
import tbcontrol.symbolic

In [19]:
s = sympy.symbols('s')

In [20]:
A, B, C, D = [sympy.Matrix(m) for m in [G2ss.A, G2ss.B, G2ss.C, G2ss.D]]

In [21]:
A, B, C, D

(Matrix([[-1.0]]), Matrix([[1.0]]), Matrix([[1.0]]), Matrix([[0.0]]))

In [22]:
G = tbcontrol.symbolic.ss2tf(A, B, C, D, s)
G

Matrix([[1.0/(s + 1.0)]])

Note that `ss2tf` returns a sympy Matrix. To get the SISO result, we need to index into the matrix:

In [23]:
G[0, 0]

1.0/(s + 1.0)

## Analysis

Notice that the roots of the characteristic function correspond with the eigenvalues of the A matrix. The numerator and denominator of control transfer functions are stored as lists of lists to accomodate MIMO systems.

In [24]:
Gtf.pole()

array([-1.])

In [25]:
numpy.roots(Gtf.den[0][0])

array([-1.])

In [26]:
numpy.linalg.eig(Gss.A)

(array([-1.]), array([[1.]]))